# Импорт библиотек

In [1]:
import mlflow
from mlflow.tracking import MlflowClient

import os

import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from transformers import pipeline
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

from datasets import Dataset

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import *

import joblib
import cloudpickle
import pickle

import nltk
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

from nltk.corpus import stopwords
from functools import lru_cache

from pymorphy3 import MorphAnalyzer
import re
import emoji
import torch

c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Smart\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Настройка конфигураций

In [ ]:
from omegaconf import OmegaConf
import os

def load_config(config_name):
    """Загрузка конфигурации для конкретного эксперимента"""
    config_path = f"configs/{config_name}.yml"
    
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Конфигурационный файл {config_path} не найден")
    
    cfg = OmegaConf.load(config_path)
    
    if 'MLFLOW_TRACKING_URI' in os.environ:
        cfg.mlflow.tracking_uri = os.environ['MLFLOW_TRACKING_URI']
    
    return cfg

# Эксперименты с датасетами

## Первый эксперимент (Лемматизация, удаление знаков пунктуации)

In [ ]:
#!   mlflow server --host 127.0.0.1 --port 8080

In [ ]:
cfg = load_config("base")

mlflow.set_tracking_uri(cfg.mlflow.tracking_uri)

print(f"Tracking URI: {cfg.mlflow.tracking_uri}")

Tracking URI: http://127.0.0.1:8080


In [ ]:
cfg = load_config("preprocess_first")

analyzer = MorphAnalyzer(lang='ru')

stop_words = nltk.corpus.stopwords.words('russian')
stop_words_cleaned = [
    w for w in stop_words
    if w not in cfg.preprocess.keep_words
]

@lru_cache(maxsize=cfg.preprocess.lru_cache_size)
def lemmatization(text):
    return analyzer.parse(text)[0].normal_form

def preprocess_text(text):
    text = re.sub(cfg.preprocess.regex.remove_newlines, " ", text)

    text = re.sub(cfg.preprocess.regex.fix_mistyped_n, r"\1", text)

    text = re.sub(cfg.preprocess.regex.remove_symbols, "", text)

    text = re.sub(cfg.preprocess.regex.collapse_spaces, " ", text).strip()

    tokens = nltk.word_tokenize(text)
    result = []

    for token in tokens:
        if token not in stop_words_cleaned:
            result.append(lemmatization(token))

    return " ".join(result)

test_text = "Всем привет! Какое же неприятное место, нет?\n"
processed = preprocess_text(test_text)
print(f"Исходный: {test_text}")
print(f"Обработанный: {processed}")

Исходный: Всем привет! Какое же неприятное место, нет?

Обработанный: весь привет какой неприятный место нет


In [ ]:
with mlflow.start_run(run_name="First dataset"):

    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)

    annotation_dfs = [
        pd.read_json(path) for path in cfg.preprocess.input_files
    ]

    df_annotations = pd.concat(annotation_dfs)
    
    df = pd.DataFrame(columns=["span", "label"])

    for mark in df_annotations['aspect_sentiment']:
        for entry in mark:
            span = entry['text']
            label = entry['labels'][0]
            df.loc[len(df)] = [span, label]

    df['span'] = df['span'].apply(preprocess_text)

    display(df)

    df.to_csv(cfg.preprocess.output.dataset_csv, index=False)

    mlflow.log_artifact(cfg.preprocess.output.dataset_csv, "datasets")

    with open(cfg.preprocess.output.preprocess_pickle, "wb") as f:
        cloudpickle.dump(preprocess_text, f)

    mlflow.log_artifact(cfg.preprocess.output.preprocess_pickle, "functions")
    

,span,label
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет любить новогодний...,ПАЧКА_POSITIVE
2504,чипсы довольно жирный не критично заметно,ТЕКСТУРА_NEUTRAL
2505,сам чипсы не идеально круглый,ТЕКСТУРА_NEUTRAL
2506,красивый золотистый нет пригорелый поломать ми...,ТЕКСТУРА_POSITIVE


🏃 View run First dataset at: http://127.0.0.1:8080/#/experiments/0/runs/8690112afeac4c7faf0867c5d30d2dd9
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Второй эксперимент (Простая очистка от знаков препинания и изменение эмодзи)

In [ ]:
from functools import partial

cfg = load_config('preprocess_second')

def clean_text_only(text, cfg=None):
    """
    Очистка текста с поддержкой конфигурации
    
    Args:
        text: Входной текст
        cfg: Конфигурация (опционально)
    """
    if not isinstance(text, str):
        return ""
    
    lowercase = getattr(cfg, 'clean_only', {}).get('lowercase', True) if cfg else True
    replace_emoji = getattr(cfg, 'clean_only', {}).get('replace_emoji', True) if cfg else True
    remove_punctuation = getattr(cfg, 'clean_only', {}).get('remove_punctuation', True) if cfg else True
    remove_special_chars = getattr(cfg, 'clean_only', {}).get('remove_special_chars', True) if cfg else True
    collapse_spaces = getattr(cfg, 'clean_only', {}).get('collapse_spaces', True) if cfg else True
    
    if lowercase:
        text = text.lower()
    
    if replace_emoji:
        text = emoji.demojize(text, delimiters=(" ", " "))
    
    if remove_special_chars:
        text = re.sub(r'[\n\r\t]', ' ', text)
    
    if remove_punctuation:
        text = re.sub(r'[^\w\s]', ' ', text)
    
    if collapse_spaces:
        text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def clean_text_only_legacy(text):
    """Легаси версия для обратной совместимости"""
    return clean_text_only(text)


with mlflow.start_run(run_name='Second dataset'):
    
    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)

    annotation_dfs = [
        pd.read_json(path) for path in cfg.preprocess.input_files
    ]

    df_annotations = pd.concat(annotation_dfs)
    
    df = pd.DataFrame(columns=["span", "label"])

    for mark in df_annotations['aspect_sentiment']:
        for entry in mark:
            span = entry['text']
            label = entry['labels'][0]
            df.loc[len(df)] = [span, label]

    df['span'] = df['span'].apply(lambda x: clean_text_only(x, cfg))
    
    display(df)

    df.to_csv(cfg.preprocess.output.dataset_csv, index=False)

    mlflow.log_artifact(cfg.preprocess.output.dataset_csv, "datasets")

    clean_text_with_config = partial(clean_text_only, cfg=cfg)
    with open(cfg.preprocess.output.preprocess_pickle, "wb") as f:
        cloudpickle.dump(clean_text_with_config, f)

    mlflow.log_artifact(cfg.preprocess.output.preprocess_pickle, "functions")

,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет я люблю новогодне...,ПАЧКА_POSITIVE
2504,чипсы довольно жирные не критично но заметно,ТЕКСТУРА_NEUTRAL
2505,сами чипсы не идеально круглые,ТЕКСТУРА_NEUTRAL
2506,красивые золотистые нет пригорелых поломанных ...,ТЕКСТУРА_POSITIVE


🏃 View run Second dataset at: http://127.0.0.1:8080/#/experiments/0/runs/ba68188717234d8ea8f01745d4816ea8
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Третий эксперимент (Аугментация данных для минорных классов)

In [8]:
with mlflow.start_run(run_name="Third dataset"):

    cfg = load_config("preprocess_third")

    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)


    annotation_dfs = [pd.read_json(path) for path in cfg.preprocess.input_files]
    df_annotations = pd.concat(annotation_dfs, ignore_index=True)


    df = pd.DataFrame(columns=["span", "label"])

    for mark in df_annotations['aspect_sentiment']:
        for entry in mark:
            span = entry['text']
            label = entry['labels'][0]
            df.loc[len(df)] = [span, label]


    class_counts = df['label'].value_counts()
    minority_classes = class_counts[class_counts < class_counts.mean()].index.tolist()

    print(f"Minority classes: {minority_classes}")


    import random

    synonyms = cfg.preprocess.augmentation.synonyms
    n_variants = cfg.preprocess.augmentation.n_variants

    def simple_augmentation(text):
        augmented = []
        for _ in range(n_variants):
            words = text.split()
            new_words = []
            for w in words:
                wl = w.lower()
                if wl in synonyms and random.random() > 0.7:
                    new_words.append(random.choice(synonyms[wl]))
                else:
                    new_words.append(w)
            augmented.append(" ".join(new_words))
        return augmented


    augmented_data = []
    for label in minority_classes:
        samples = df[df['label'] == label]
        for _, row in samples.iterrows():
            for aug_text in simple_augmentation(row["span"]):
                augmented_data.append({"span": aug_text, "label": label})

    df_augmented = pd.DataFrame(augmented_data)


    df_extended = pd.concat([df, df_augmented], ignore_index=True)


    output_path = cfg.preprocess.output.dataset_csv
    df_extended.to_csv(output_path, index=False)

    mlflow.log_artifact(output_path, "datasets")

    print(f"Сохранен датасет с аугментацией: {output_path}")


Minority classes: ['ПАЧКА_POSITIVE', 'ТЕКСТУРА_NEUTRAL', 'ПАЧКА_NEUTRAL', 'ТЕКСТУРА_NEGATIVE', 'ПАЧКА_NEGATIVE']
Сохранен датасет с аугментацией: third_experiment_dataset.csv
🏃 View run Third dataset at: http://127.0.0.1:8080/#/experiments/0/runs/327f7f34ba914fce83ca6c0e787e8f15
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


# Эксперементирование с моделями

## Первый эксперимент (Простая модель)

Будем обучать простую модель: "Наивный Байесовский Классификатор" из `Sklearn`, будем проводить тест на 3 версиях датасета и поймем, какая модель лучше себя покажет при работе с тем или иным датасетом

### Первый датасет

In [ ]:
cfg = load_config("naive_bayes_first")

with mlflow.start_run(run_name='first_model_experiment'):

    mlflow.set_tag('NaiveBayes', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id

    # Используем пути из конфига
    full_dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
    
    try:
        dataframe_path = client.download_artifacts(first_dataset_latest_run_id, full_dataset_path)
        df = pd.read_csv(dataframe_path)
        print(f"✅ Успешно загружен: {full_dataset_path}")
    except Exception as e:
        print(f"❌ Не удалось загрузить {full_dataset_path}: {e}")

        try:
            dataframe_path = client.download_artifacts(first_dataset_latest_run_id, cfg.data.dataset_file)
            df = pd.read_csv(dataframe_path)
            print(f"✅ Успешно загружен: {cfg.data.dataset_file}")
        except:
            raise ValueError(f"Не удалось загрузить датасет: {full_dataset_path}")
    
    display(df)

    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectorizer = TfidfVectorizer(
        lowercase=cfg.vectorizer.lowercase,
        analyzer=cfg.vectorizer.analyzer,
        max_features=cfg.vectorizer.max_features,
        ngram_range=tuple(cfg.vectorizer.ngram_range),
        min_df=cfg.vectorizer.min_df,
        max_df=cfg.vectorizer.max_df
    )
    
    encoder = LabelEncoder()

    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST
    # =====================================================================================================================================

    df_train, df_test = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    X_train = df_train['span']
    X_train = vectorizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)

    X_test = df_test['span']
    X_test = vectorizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)

    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================


    mlflow.sklearn.autolog(disable=True)
    
    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f'f1-score равен {f1:.4f}')
    print(f'accuracy-score равен {accuracy:.4f}')

    joblib.dump(model, cfg.model.artifacts.model)
    joblib.dump(vectorizer, cfg.model.artifacts.vectorizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)
    
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.vectorizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')
    
    mlflow.log_metrics({
        'f1_score': f1,
        'accuracy': accuracy
    })

    mlflow.log_params({
        'vectorizer_max_features': cfg.vectorizer.max_features,
        'vectorizer_ngram_range': str(cfg.vectorizer.ngram_range),
        'test_size': cfg.training.test_size
    })

    print("Модель успешно обучена и залогирована!")

✅ Успешно загружен: datasets/first_experiment_dataset.csv


,span,label
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет любить новогодний...,ПАЧКА_POSITIVE
2504,чипсы довольно жирный не критично заметно,ТЕКСТУРА_NEUTRAL
2505,сам чипсы не идеально круглый,ТЕКСТУРА_NEUTRAL
2506,красивый золотистый нет пригорелый поломать ми...,ТЕКСТУРА_POSITIVE


f1-score равен 0.3869
accuracy-score равен 0.4422
Модель успешно обучена и залогирована!
🏃 View run first_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/64117c4a566b416fb2c5adeafb5de204
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_bayes_first")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


vectorizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.vectorizer}"
bayes_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.bayes}" 
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

try:
    vectorizer_file = client.download_artifacts(latest_run_model_id, vectorizer_path)
    bayes_file = client.download_artifacts(latest_run_model_id, bayes_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("✅ Модели успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")

    try:
        vectorizer_file = client.download_artifacts(latest_run_model_id, cfg_inference.model.vectorizer)
        bayes_file = client.download_artifacts(latest_run_model_id, cfg_inference.model.bayes)
        encoder_file = client.download_artifacts(latest_run_model_id, cfg_inference.model.encoder)
        print("✅ Модели загружены (fallback)")
    except:
        raise ValueError("Не удалось загрузить модели")

# ===========================================================================================
# ПОЛУЧЕНИЕ ФУНКЦИИ ПРЕПРОЦЕССИНГА
# ===========================================================================================


latest_run_dataset = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.preprocess.run_name}"',
    order_by=['attributes.end_time desc']
)

latest_run_dataset_id = latest_run_dataset[0].info.run_id

try:
    art_loc = client.download_artifacts(latest_run_dataset_id, cfg_inference.preprocess.artifact_path)
    print("✅ Функция препроцессинга загружена")
except Exception as e:
    print(f"❌ Ошибка загрузки функции: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================


vectorizer = joblib.load(vectorizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)


with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

preprocessed_text = preprocess_func(text)
done_text = vectorizer.transform([preprocessed_text])
label = bayes.predict(done_text)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(f'Предсказанная метка: {encoder.inverse_transform(label)[0]}')

Run ID модели: 64117c4a566b416fb2c5adeafb5de204


✅ Модели успешно загружены


✅ Функция препроцессинга загружена
Исходный текст: "Меня привлекло такое сочетание"
Предобработанный текст: "я привлечь такой сочетание"
Предсказанная метка: ВКУС_POSITIVE


### Второй датасет

In [11]:

cfg = load_config("naive_bayes_second")

with mlflow.start_run(run_name='second_model_experiment'):

    mlflow.set_tag('NaiveBayes', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    first_dataset_latest_run = dataset_runs[0]
    first_dataset_latest_run_id = first_dataset_latest_run.info.run_id


    try:
        dataframe_path = client.download_artifacts(first_dataset_latest_run_id, "datasets/second_experiment_dataset.csv")
        df = pd.read_csv(dataframe_path)
        print("✅ Датасет загружен напрямую")
    except:
        try:
            files = client.list_artifacts(first_dataset_latest_run_id, cfg.data.dataset_path)
            dataframe = files[0].path
            dataframe_path = client.download_artifacts(first_dataset_latest_run_id, dataframe)
            df = pd.read_csv(dataframe_path)
            print("✅ Датасет загружен через list_artifacts")
        except:
            raise ValueError("Не удалось загрузить датасет")

    display(df)

    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectorizer = TfidfVectorizer(
        lowercase=cfg.vectorizer.lowercase,
        analyzer=cfg.vectorizer.analyzer,
        max_features=cfg.vectorizer.max_features,
        ngram_range=tuple(cfg.vectorizer.ngram_range),
        min_df=cfg.vectorizer.min_df,
        max_df=cfg.vectorizer.max_df
    )
    
    encoder = LabelEncoder()

    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST
    # =====================================================================================================================================

    df_train, df_test = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    X_train = df_train['span']
    X_train = vectorizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)

    X_test = df_test['span']
    X_test = vectorizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)

    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog(disable=True)

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f'f1-score равен {f1:.4f}')
    print(f'accuracy-score равен {accuracy:.4f}')

    joblib.dump(model, cfg.model.artifacts.model)
    joblib.dump(vectorizer, cfg.model.artifacts.vectorizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)
    
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.vectorizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')
    
    mlflow.log_metrics({
        'f1_score': f1,
        'accuracy': accuracy
    })

    mlflow.log_params({
        'vectorizer_max_features': cfg.vectorizer.max_features,
        'vectorizer_ngram_range': str(cfg.vectorizer.ngram_range),
        'test_size': cfg.training.test_size
    })

    print("Вторая модель успешно обучена и залогирована!")

✅ Датасет загружен напрямую


,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет я люблю новогодне...,ПАЧКА_POSITIVE
2504,чипсы довольно жирные не критично но заметно,ТЕКСТУРА_NEUTRAL
2505,сами чипсы не идеально круглые,ТЕКСТУРА_NEUTRAL
2506,красивые золотистые нет пригорелых поломанных ...,ТЕКСТУРА_POSITIVE


f1-score равен 0.3675
accuracy-score равен 0.4223
Вторая модель успешно обучена и залогирована!
🏃 View run second_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/2414d58064a146a69e694625b0041c82
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_bayes_second")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"', 
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


vectorizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.vectorizer}"
bayes_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.bayes}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    vectorizer_file = client.download_artifacts(latest_run_model_id, vectorizer_path)
    bayes_file = client.download_artifacts(latest_run_model_id, bayes_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("Модели успешно загружены")
except Exception as e:
    print(f"Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ПОЛУЧЕНИЕ ФУНКЦИИ ПРЕПРОЦЕССИНГА
# ===========================================================================================


latest_run_dataset = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.preprocess.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_dataset:
    raise ValueError(f"Не найден run: {cfg_inference.preprocess.run_name}")

latest_run_dataset_id = latest_run_dataset[0].info.run_id

print("Загружаем функцию препроцессинга...")
try:
    art_loc = client.download_artifacts(latest_run_dataset_id, cfg_inference.preprocess.artifact_path)
    print("Функция препроцессинга загружена")
except Exception as e:
    print(f"Ошибка загрузки функции: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
vectorizer = joblib.load(vectorizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

print("Загружаем функцию препроцессинга...")
with open(art_loc, 'rb') as f:
    preprocess_func = cloudpickle.load(f)

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

print("Выполняем предсказание...")
preprocessed_text = preprocess_func(text)
done_text = vectorizer.transform([preprocessed_text])
label = bayes.predict(done_text)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(f'Предсказанная метка: {encoder.inverse_transform(label)[0]}')

Run ID модели: 2414d58064a146a69e694625b0041c82
Загружаем модели...


✅ Модели успешно загружены
Загружаем функцию препроцессинга...


✅ Функция препроцессинга загружена
Загружаем модели в память...
Загружаем функцию препроцессинга...
Выполняем предсказание...
Исходный текст: "Меня не привлекло такое сочетание 😭"
Предобработанный текст: "меня не привлекло такое сочетание loudly_crying_face"
Предсказанная метка: ВКУС_NEGATIVE


### Третий датасет

In [ ]:
cfg = load_config("naive_bayes_third")

with mlflow.start_run(run_name='third_model_experiment'):

    mlflow.set_tag('NaiveBayes', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"Найден run с датасетом: {dataset_run_id}")


    try:
        dataframe_path = client.download_artifacts(dataset_run_id, f"datasets/{cfg.data.dataset_file}")
        df = pd.read_csv(dataframe_path)
        print("Датасет загружен напрямую")
    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/datasets/{cfg.data.dataset_file}",
            "third_experiment_dataset.csv"
        ]
        
        for path in alternative_paths:
            try:
                dataframe_path = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(dataframe_path)
                print(f"Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df)

    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    vectorizer = TfidfVectorizer(
        lowercase=cfg.vectorizer.lowercase,
        analyzer=cfg.vectorizer.analyzer,
        max_features=cfg.vectorizer.max_features,
        ngram_range=tuple(cfg.vectorizer.ngram_range),
        min_df=cfg.vectorizer.min_df,
        max_df=cfg.vectorizer.max_df
    )
    
    encoder = LabelEncoder()

    # =====================================================================================================================================
    #                                         РАЗБИЕНИЕ НА TRAIN/TEST
    # =====================================================================================================================================

    df_train, df_test = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    X_train = df_train['span']
    X_train = vectorizer.fit_transform(X_train)

    y_train = df_train['label']
    y_train = encoder.fit_transform(y_train)

    X_test = df_test['span']
    X_test = vectorizer.transform(X_test)

    y_test = df_test['label']
    y_test = encoder.transform(y_test)

    # =====================================================================================================================================
    #                                         ОБУЧЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    mlflow.sklearn.autolog(disable=True)

    model = MultinomialNB()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)

    print(f'f1-score равен {f1:.4f}')
    print(f'accuracy-score равен {accuracy:.4f}')

    joblib.dump(model, cfg.model.artifacts.model)
    joblib.dump(vectorizer, cfg.model.artifacts.vectorizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)
    
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.vectorizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')
    
    mlflow.log_metrics({
        'f1_score': f1,
        'accuracy': accuracy
    })

    mlflow.log_params({
        'vectorizer_max_features': cfg.vectorizer.max_features,
        'vectorizer_ngram_range': str(cfg.vectorizer.ngram_range),
        'test_size': cfg.training.test_size
    })

    print("Третья модель успешно обучена и залогирована!")

✅ Найден run с датасетом: 327f7f34ba914fce83ca6c0e787e8f15


✅ Датасет загружен напрямую


,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,"Я ел эти чипсы очень долго, еще года 2 назад с...",O
2,"хотелось купить очень много, но в последнее вр...",O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал НАМНОГО острее обычного,ВКУС_NEGATIVE
...,...,...
4015,"с чипсами всегда стоит помнить, что две трети ...",ПАЧКА_NEGATIVE
4016,"Пластиковая крышка попалась очень тугая, еле-е...",ПАЧКА_NEGATIVE
4017,"Пластиковая крышка попалась очень тугая, еле-е...",ПАЧКА_NEGATIVE
4018,Наполнеине пачки типичное для данного производ...,ПАЧКА_NEGATIVE


f1-score равен 0.5517
accuracy-score равен 0.5958
Третья модель успешно обучена и залогирована!
🏃 View run third_model_experiment at: http://127.0.0.1:8080/#/experiments/0/runs/b5cc3981bcdd47d2ae00e73f194e41d0
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатоы

In [ ]:
cfg_inference = load_config("inference_bayes_third")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


vectorizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.vectorizer}"
bayes_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.bayes}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    vectorizer_file = client.download_artifacts(latest_run_model_id, vectorizer_path)
    bayes_file = client.download_artifacts(latest_run_model_id, bayes_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("Модели успешно загружены")
except Exception as e:
    print(f"Ошибка загрузки моделей: {e}")
    raise


# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
vectorizer = joblib.load(vectorizer_file)
bayes = joblib.load(bayes_file)
encoder = joblib.load(encoder_file)

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

print("Выполняем предсказание...")
done_text = vectorizer.transform([text])
label = bayes.predict(done_text)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предобработанный текст: "{preprocessed_text}"')
print(f'Предсказанная метка: {encoder.inverse_transform(label)[0]}')
print('=' * 100)

Run ID модели: b5cc3981bcdd47d2ae00e73f194e41d0
Загружаем модели...


✅ Модели успешно загружены
Загружаем модели в память...
Выполняем предсказание...
Исходный текст: "Меня привлекло такое сочетание"
Предобработанный текст: "меня не привлекло такое сочетание loudly_crying_face"
Предсказанная метка: ПАЧКА_POSITIVE


### Итог

In [15]:
results = pd.DataFrame({
    'Версия': ['Версия 1', 'Версия 2', 'Версия 3'],
    'F1-Score': [0.363707, 0.344306, 0.344139],
    'Accuracy': [0.413043, 0.391304, 0.391304],
    'Описание': ['Лемматизация, удаление знаков пунктуации', 'Очистка от знаков препинания и emoji to text', 'Никакой очистки']
})

results

,Версия,F1-Score,Accuracy,Описание
0,Версия 1,0.363707,0.413043,"Лемматизация, удаление знаков пунктуации"
1,Версия 2,0.344306,0.391304,Очистка от знаков препинания и emoji to text
2,Версия 3,0.344139,0.391304,Никакой очистки


## Второй эксперимент (Keras)

### Первый датасет

In [ ]:

cfg = load_config("neural_network_first")

with mlflow.start_run(run_name='first_experiment_neural_network'):
    
    mlflow.set_tag('LSTM', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    try:
        dataframe_path = client.download_artifacts(dataset_run_id, f"{cfg.data.dataset_path}/{cfg.data.dataset_file}")
        df = pd.read_csv(dataframe_path)
        print("Датасет загружен напрямую")
    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        try:
            files = client.list_artifacts(dataset_run_id, cfg.data.dataset_path)
            dataframe = files[0].path
            dataframe_path = client.download_artifacts(dataset_run_id, dataframe)
            df = pd.read_csv(dataframe_path)
            print("Датасет загружен через list_artifacts")
        except Exception as e2:
            print(f"Ошибка загрузки через list_artifacts: {e2}")
            alternative_paths = [
                cfg.data.dataset_file,
                f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
                "First_version.csv"
            ]
            
            for path in alternative_paths:
                try:
                    dataframe_path = client.download_artifacts(dataset_run_id, path)
                    df = pd.read_csv(dataframe_path)
                    print(f"✅ Датасет загружен: {path}")
                    break
                except:
                    continue
            else:
                raise ValueError("Не удалось загрузить датасет")

    display(df)
    
    # =====================================================================================================================================
    #                                              ВЕКТОРИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    mlflow.tensorflow.autolog()

    tokenizer = Tokenizer(
        num_words=cfg.tokenizer.num_words,
        oov_token=cfg.tokenizer.oov_token,
        filters=cfg.tokenizer.filters,
        lower=cfg.tokenizer.lower,
        split=cfg.tokenizer.split,
        char_level=cfg.tokenizer.char_level
    )

    tokenizer.fit_on_texts(df['span'])

    df_train, df_temp = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['label']
    )

    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=df_temp['label']
    )

    X_train_vec = tokenizer.texts_to_sequences(df_train['span'])
    X_test_vec = tokenizer.texts_to_sequences(df_test['span'])
    X_val_vec = tokenizer.texts_to_sequences(df_val['span'])

    max_len_text = 0
    for i in df['span']:
        max_len_text = max(max_len_text, len(i.split(' ')))
    print(f"Максимальная длина текста: {max_len_text}")


    X_train_pad = pad_sequences(X_train_vec, max_len_text, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_vec, max_len_text, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_vec, max_len_text, padding='post', truncating='post')

    # =====================================================================================================================================
    #                                              ПОДГОТОВКА ТАРГЕТОВ
    # =====================================================================================================================================

    encoder = LabelEncoder()

    y_train = encoder.fit_transform(df_train['label'])
    y_test = encoder.transform(df_test['label'])
    y_val = encoder.transform(df_val['label'])

    # =====================================================================================================================================
    #                                              СОЗДАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    model = Sequential()

    model.add(Embedding(
        input_dim=cfg.tokenizer.num_words, 
        output_dim=cfg.model.embedding_dim, 
        input_length=max_len_text
    ))

    model.add(Bidirectional(LSTM(
        cfg.model.lstm_units, 
        dropout=0.2, 
        recurrent_dropout=0.3
    )))

    model.add(Dropout(cfg.model.dropout_rate))

    model.add(Dense(cfg.model.dense_units, activation='softmax'))

    model.compile(
        optimizer='adam', 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )

    model.fit(
        X_train_pad, 
        y_train, 
        batch_size=cfg.training.batch_size, 
        epochs=cfg.training.epochs, 
        validation_data=(X_val_pad, y_val)
    )

    test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    model.summary()

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    pred_proba = model.predict(X_test_pad)
    pred_class = np.argmax(pred_proba, axis=1)
    
    f1 = f1_score(pred_class, y_test, average='weighted')
    accuracy = accuracy_score(pred_class, y_test)

    print(f'f1-score у модели равен {f1:.4f}')
    print(f'accuracy у модели равен {accuracy:.4f}')

    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)
    mlflow.log_metric('test_loss', test_loss)

    model.save("LSTM_ver_1.keras")

    with open(cfg.model.artifacts.tokenizer, 'wb') as f:
        cloudpickle.dump(tokenizer, f)
    
    with open(cfg.model.artifacts.encoder, 'wb') as f:
        cloudpickle.dump(encoder, f)

    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.tokenizer, 'models') 
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')

    mlflow.log_params({
        'num_words': cfg.tokenizer.num_words,
        'embedding_dim': cfg.model.embedding_dim,
        'lstm_units': cfg.model.lstm_units,
        'dropout_rate': cfg.model.dropout_rate,
        'batch_size': cfg.training.batch_size,
        'epochs': cfg.training.epochs
    })

    print("Нейронная сеть успешно обучена и залогирована!")

✅ Найден run с датасетом: 8690112afeac4c7faf0867c5d30d2dd9


✅ Датасет загружен напрямую


,span,label
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет любить новогодний...,ПАЧКА_POSITIVE
2504,чипсы довольно жирный не критично заметно,ТЕКСТУРА_NEUTRAL
2505,сам чипсы не идеально круглый,ТЕКСТУРА_NEUTRAL
2506,красивый золотистый нет пригорелый поломать ми...,ТЕКСТУРА_POSITIVE


Максимальная длина текста: 73


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025/11/26 01:15:28 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: Sequential model 'sequential' has no defined input shape yet.


...

 Total params: 1,068,320 (4.08 MB)

 Trainable params: 356,106 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 712,214 (2.72 MB)

8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 270ms/step
f1-score у модели равен 0.4730
accuracy у модели равен 0.4701
✅ Нейронная сеть успешно обучена и залогирована!
🏃 View run first_experiment_neural_network at: http://127.0.0.1:8080/#/experiments/0/runs/bd1c61c3551940db8aa3011106254c09
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_neural_network")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

model_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.model_file}"
tokenizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.tokenizer}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    model_file = client.download_artifacts(latest_run_model_id, model_path)
    tokenizer_file = client.download_artifacts(latest_run_model_id, tokenizer_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("Модели успешно загружены")
except Exception as e:
    print(f"Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model_keras = tf.keras.models.load_model(model_file)
encoder = joblib.load(encoder_file)
tokenizer = joblib.load(tokenizer_file)

print("Все модели загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

print("Выполняем предсказание...")

tokenized_text = tokenizer.texts_to_sequences([text])

max_len = model_keras.input_shape[1]
print(f"Максимальная длина последовательности: {max_len}")

padded_text = pad_sequences(tokenized_text, maxlen=max_len, padding='post', truncating='post')

pred = model_keras.predict(padded_text)

predicted_class_ind = np.argmax(pred, axis=1)
predicted_class = encoder.inverse_transform(predicted_class_ind)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_class[0]}')
print(f'Вероятности по классам:')
for i, prob in enumerate(pred[0]):
    class_name = encoder.inverse_transform([i])[0]
    print(f'  {class_name}: {prob:.4f}')
print('=' * 100)

Run ID модели: bd1c61c3551940db8aa3011106254c09
Загружаем модели...


✅ Модели успешно загружены
Загружаем модели в память...
✅ Все модели загружены в память
Выполняем предсказание...
Максимальная длина последовательности: 73
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Исходный текст: "В целом, чипсы Ашан Красная птица Барбекю вполне съедобные"
Предсказанная метка: ВКУС_NEUTRAL
Вероятности по классам:
  O: 0.0332
  ВКУС_NEGATIVE: 0.0001
  ВКУС_NEUTRAL: 0.9660
  ВКУС_POSITIVE: 0.0002
  ПАЧКА_NEGATIVE: 0.0001
  ПАЧКА_NEUTRAL: 0.0002
  ПАЧКА_POSITIVE: 0.0000
  ТЕКСТУРА_NEGATIVE: 0.0000
  ТЕКСТУРА_NEUTRAL: 0.0001
  ТЕКСТУРА_POSITIVE: 0.0002


### Второй датасет

In [ ]:

cfg = load_config("neural_network_second")

with mlflow.start_run(run_name='second_experiment_neural_network'):
    
    mlflow.set_tag('CNN', cfg.model.version)

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    client = MlflowClient()

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art)
        print(f"Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "Second_version.csv"
        ]
        
        for path in alternative_paths:
            try:
                art = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df)

    # =====================================================================================================================================
    #                                         ТОКЕНИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    mlflow.tensorflow.autolog()

    tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
    tokenizer = Tokenizer(**tokenizer_config)
    
    tokenizer.fit_on_texts(df['span'])

    df_train, df_temp = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state
    )
    
    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state
    )

    X_train_vec = tokenizer.texts_to_sequences(df_train['span'])
    X_test_vec = tokenizer.texts_to_sequences(df_test['span'])
    X_val_vec = tokenizer.texts_to_sequences(df_val['span'])

    X_train_pad = pad_sequences(X_train_vec, maxlen=cfg.training.max_sequence_length, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_vec, maxlen=cfg.training.max_sequence_length, padding='post', truncating='post')
    X_val_pad = pad_sequences(X_val_vec, maxlen=cfg.training.max_sequence_length, padding='post', truncating='post')

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ТАРГЕТОВ
    # =====================================================================================================================================

    encoder = LabelEncoder()

    y_train = encoder.fit_transform(df_train['label'])
    y_test = encoder.transform(df_test['label'])
    y_val = encoder.transform(df_val['label'])

    # =====================================================================================================================================
    #                                         СОЗДАНИЕ МОДЕЛИ CNN
    # =====================================================================================================================================

    model = Sequential()

    model.add(Embedding(
        input_dim=cfg.tokenizer.num_words, 
        output_dim=cfg.model.embedding_dim, 
        input_length=cfg.training.max_sequence_length
    ))

    model.add(Conv1D(
        filters=cfg.model.conv_filters,
        kernel_size=cfg.model.conv_kernel_size,
        activation=cfg.model.conv_activation
    ))

    model.add(Dropout(cfg.model.dropout_rate_1))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(cfg.model.dense_units_1, activation=cfg.model.dense_activation_1))

    model.add(Dropout(cfg.model.dropout_rate_2))

    model.add(Dense(cfg.model.dense_units_2, activation=cfg.model.dense_activation_2))

    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer='adam', 
        metrics=['accuracy']
    )

    model.fit(
        X_train_pad, 
        y_train, 
        batch_size=cfg.training.batch_size, 
        epochs=cfg.training.epochs, 
        validation_data=(X_val_pad, y_val)
    )

    model.summary()

    test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    pred = model.predict(X_test_pad)
    pred_class = np.argmax(pred, axis=1)

    f1 = f1_score(pred_class, y_test, average='weighted')
    accuracy = accuracy_score(pred_class, y_test)

    print(f'f1-score у модели равен {f1:.4f}')
    print(f'accuracy у модели равен {accuracy:.4f}')

    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('test_accuracy', test_accuracy)
    mlflow.log_metric('test_loss', test_loss)

    model.save(cfg.model.artifacts.model)

    joblib.dump(tokenizer, cfg.model.artifacts.tokenizer)
    joblib.dump(encoder, cfg.model.artifacts.encoder)

    mlflow.log_artifact(cfg.model.artifacts.model, 'models')
    mlflow.log_artifact(cfg.model.artifacts.tokenizer, 'models')
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'models')

    mlflow.log_params({
        'num_words': cfg.tokenizer.num_words,
        'embedding_dim': cfg.model.embedding_dim,
        'conv_filters': cfg.model.conv_filters,
        'conv_kernel_size': cfg.model.conv_kernel_size,
        'dropout_rate_1': cfg.model.dropout_rate_1,
        'dropout_rate_2': cfg.model.dropout_rate_2,
        'dense_units_1': cfg.model.dense_units_1,
        'batch_size': cfg.training.batch_size,
        'epochs': cfg.training.epochs,
        'max_sequence_length': cfg.training.max_sequence_length
    })

    print("✅ CNN модель успешно обучена и залогирована!")

✅ Найден run с датасетом: ba68188717234d8ea8f01745d4816ea8


✅ Датасет загружен: datasets/second_experiment_dataset.csv


,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет я люблю новогодне...,ПАЧКА_POSITIVE
2504,чипсы довольно жирные не критично но заметно,ТЕКСТУРА_NEUTRAL
2505,сами чипсы не идеально круглые,ТЕКСТУРА_NEUTRAL
2506,красивые золотистые нет пригорелых поломанных ...,ТЕКСТУРА_POSITIVE


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025/11/26 01:21:32 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: Sequential model 'sequential_1' has no defined input shape yet.


Epoch 1/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.1939 - loss: 2.2327 - val_accuracy: 0.1753 - val_loss: 2.2185
Epoch 2/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2094 - loss: 2.1531 - val_accuracy: 0.1873 - val_loss: 2.1567
Epoch 3/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2662 - loss: 2.0085 - val_accuracy: 0.2829 - val_loss: 2.0674
Epoch 4/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3988 - loss: 1.7389 - val_accuracy: 0.4104 - val_loss: 1.9122
Epoch 5/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5040 - loss: 1.4663 - val_accuracy: 0.4183 - val_loss: 1.8126
Epoch 6/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5897 - loss: 1.2436 - val_accuracy: 0.4422 - val_loss: 1.7648
Epoch 7/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6441 - loss: 1.0564 - val_accuracy: 0.3984 - val_loss: 1.7366
Epoch 8/40
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6949 - loss: 0.9225 - val_accuracy: 0

...

 Total params: 404,624 (1.54 MB)

 Trainable params: 134,874 (526.85 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 269,750 (1.03 MB)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4024 - loss: 3.4085 
Test Accuracy: 0.4024
Test Loss: 3.4085
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/stepWARNING:tensorflow:5 out of the last 17 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000028E1C6F39C0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
f1-score у модели равен 0.4025
accuracy у модели равен 0.4024
✅ CNN модель успешно обу

#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_neural_network_second")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

model_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.model_file}"
tokenizer_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.tokenizer}"
encoder_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.encoder}"

print("Загружаем модели...")
try:
    model_file = client.download_artifacts(latest_run_model_id, model_path)
    tokenizer_file = client.download_artifacts(latest_run_model_id, tokenizer_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    print("Модели успешно загружены")
except Exception as e:
    print(f"Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model_keras = tf.keras.models.load_model(model_file)
encoder = joblib.load(encoder_file)
tokenizer = joblib.load(tokenizer_file)

print("Все модели загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================


text = cfg_inference.test_text

print("Выполняем предсказание...")


tokenized_text = tokenizer.texts_to_sequences([text])


max_len = model_keras.input_shape[1]
print(f"Максимальная длина последовательности: {max_len}")


padded_text = pad_sequences(tokenized_text, maxlen=max_len, padding='post', truncating='post')


pred = model_keras.predict(padded_text)


predicted_class_ind = np.argmax(pred, axis=1)
predicted_class = encoder.inverse_transform(predicted_class_ind)

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_class[0]}')
print(f'Вероятности по классам:')
for i, prob in enumerate(pred[0]):
    class_name = encoder.inverse_transform([i])[0]
    print(f'  {class_name}: {prob:.4f}')
print('=' * 100)

Run ID модели: 7d097ddcda574eb39cb841127b8e31f3
Загружаем модели...


✅ Модели успешно загружены
Загружаем модели в память...
✅ Все модели загружены в память
Выполняем предсказание...
Максимальная длина последовательности: 100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
Исходный текст: "В целом, чипсы Ашан Красная птица Барбекю вполне съедобные"
Предсказанная метка: ВКУС_NEUTRAL
Вероятности по классам:
  O: 0.0063
  ВКУС_NEGATIVE: 0.0010
  ВКУС_NEUTRAL: 0.7726
  ВКУС_POSITIVE: 0.0001
  ПАЧКА_NEGATIVE: 0.0600
  ПАЧКА_NEUTRAL: 0.1581
  ПАЧКА_POSITIVE: 0.0019
  ТЕКСТУРА_NEGATIVE: 0.0000
  ТЕКСТУРА_NEUTRAL: 0.0001
  ТЕКСТУРА_POSITIVE: 0.0000


### Третий датасет

In [ ]:
cfg = load_config("neural_network_third")

with mlflow.start_run(run_name='third_experiment_neural_network'):
    
    mlflow.set_tag('Bidirectional_GRU', cfg.model.version)
    client = MlflowClient()

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"✅ Найден run с датасетом: {dataset_run_id}")

    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art)
        print(f"Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "third_experiment_dataset.csv"
        ]
        
        for path in alternative_paths:
            try:
                art = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art)
                print(f"✅ Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df.head())

    # =====================================================================================================================================
    #                                         ТОКЕНИЗАЦИЯ ТЕКСТА
    # =====================================================================================================================================

    mlflow.tensorflow.autolog()
    mlflow.sklearn.autolog()

    # Создаем токенизатор с параметрами из конфига
    tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
    tokenizer = Tokenizer(**tokenizer_config)
    
    tokenizer.fit_on_texts(df['span'])

    sequences = tokenizer.texts_to_sequences(df['span'])

    # Автоматическое определение максимальной длины
    max_len = max(len(seq) for seq in sequences)
    print(f"Максимальная длина последовательности: {max_len}")

    X = pad_sequences(sequences, maxlen=max_len)

    # =====================================================================================================================================
    #                                         ПРЕОБРАЗОВАНИЕ ТАРГЕТА
    # =====================================================================================================================================

    encoder = LabelEncoder()
    y = encoder.fit_transform(df['label'])
    num_classes = len(encoder.classes_)
    
    # =====================================================================================================================================
    #                                         РАЗДЕЛЕНИЕ НА TRAIN/TEST/VAL
    # =====================================================================================================================================

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, 
        train_size=1-cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=y
    )

    X_test, X_val, y_test, y_val = train_test_split(
        X_temp, y_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=y_temp
    )

    print(f"📊 Размеры данных:")
    print(f"   Train: {X_train.shape}, {y_train.shape}")
    print(f"   Test: {X_test.shape}, {y_test.shape}")
    print(f"   Val: {X_val.shape}, {y_val.shape}")

    # =====================================================================================================================================
    #                                         СОЗДАНИЕ И ОБУЧЕНИЕ МОДЕЛИ
    # =====================================================================================================================================

    model = Sequential()

    model.add(Embedding(
        input_dim=cfg.tokenizer.num_words, 
        output_dim=cfg.model.embedding_dim
    ))

    model.add(Bidirectional(GRU(cfg.model.gru_units)))

    model.add(Dense(cfg.model.dense_units, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation="softmax"))

    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer='adam', 
        metrics=['accuracy']
    )

    model.fit(
        X_train, y_train, 
        epochs=cfg.training.epochs, 
        batch_size=cfg.training.batch_size, 
        validation_data=(X_val, y_val)
    )

    # =====================================================================================================================================
    #                                         ОЦЕНКА МОДЕЛИ
    # =====================================================================================================================================

    model.summary()

    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test Loss: {loss:.4f}')
    print(f'Test Accuracy: {accuracy:.4f}')

    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)

    f1 = f1_score(y_pred, y_test, average='weighted')
    acc_score = accuracy_score(y_pred, y_test)

    print(f'F1-score: {f1:.4f}')
    print(f'Accuracy: {acc_score:.4f}')

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ МЕТРИК
    # =====================================================================================================================================

    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', acc_score)
    mlflow.log_metric('test_loss', loss)
    mlflow.log_metric('test_accuracy', accuracy)

    # =====================================================================================================================================
    #                                         СОХРАНЕНИЕ АРТЕФАКТОВ
    # =====================================================================================================================================

    model.save(cfg.model.artifacts.model)
    mlflow.log_artifact(cfg.model.artifacts.model, 'models')

    with open(cfg.model.artifacts.tokenizer, 'wb') as f:
        pickle.dump(tokenizer, f)
    mlflow.log_artifact(cfg.model.artifacts.tokenizer, 'preprocessing')

    with open(cfg.model.artifacts.encoder, 'wb') as f:
        pickle.dump(encoder, f)
    mlflow.log_artifact(cfg.model.artifacts.encoder, 'preprocessing')

    preprocessing_info = {
        'vocab_size': len(tokenizer.word_index),
        'max_sequence_length': max_len,
        'num_classes': num_classes,
        'classes': list(encoder.classes_)
    }
    
    with open(cfg.model.artifacts.preprocessing_info, 'wb') as f:
        pickle.dump(preprocessing_info, f)
    mlflow.log_artifact(cfg.model.artifacts.preprocessing_info, 'preprocessing')

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ ПАРАМЕТРОВ
    # =====================================================================================================================================

    mlflow.log_params({
        "model_type": "Bidirectional_GRU",
        "embedding_dim": cfg.model.embedding_dim,
        "gru_units": cfg.model.gru_units,
        "dense_units": cfg.model.dense_units,
        "num_classes": num_classes,
        "vocab_size": len(tokenizer.word_index),
        "max_sequence_length": max_len,
        "batch_size": cfg.training.batch_size,
        "epochs": cfg.training.epochs
    })

    print("Bidirectional GRU модель успешно обучена и залогирована!")

✅ Найден run с датасетом: 327f7f34ba914fce83ca6c0e787e8f15


✅ Датасет загружен: datasets/third_experiment_dataset.csv


,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,"Я ел эти чипсы очень долго, еще года 2 назад с...",O
2,"хотелось купить очень много, но в последнее вр...",O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал НАМНОГО острее обычного,ВКУС_NEGATIVE


2025/11/26 01:22:32 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: Sequential model 'sequential_2' has no defined input shape yet.


📏 Максимальная длина последовательности: 85
📊 Размеры данных:
   Train: (3216, 85), (3216,)
   Test: (402, 85), (402,)
   Val: (402, 85), (402,)
Epoch 1/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 22s 158ms/step - accuracy: 0.4667 - loss: 1.5992 - val_accuracy: 0.4577 - val_loss: 2.0455
Epoch 2/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 15s 151ms/step - accuracy: 0.7848 - loss: 0.6825 - val_accuracy: 0.5746 - val_loss: 1.6586
Epoch 3/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 20s 146ms/step - accuracy: 0.8859 - loss: 0.4023 - val_accuracy: 0.6791 - val_loss: 1.1612
Epoch 4/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 16s 159ms/step - accuracy: 0.9288 - loss: 0.2556 - val_accuracy: 0.6318 - val_loss: 1.3787
Epoch 5/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 18s 181ms/step - accuracy: 0.9443 - loss: 0.1877 - val_accuracy: 0.6667 - val_loss: 1.3948
Epoch 6/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 17s 167ms/step - accuracy: 0.9608 - loss: 0.1290 - val_accuracy: 0.6517 - val_loss: 1.8094
Epoch 7/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 19s 155ms/step - accuracy: 0.

...

 Total params: 4,602,976 (17.56 MB)

 Trainable params: 1,534,282 (5.85 MB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 3,068,566 (11.71 MB)

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.7189 - loss: 1.4945
📊 Test Loss: 1.4945
📊 Test Accuracy: 0.7189
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step
🎯 F1-score: 0.7280
🎯 Accuracy: 0.7189
✅ Bidirectional GRU модель успешно обучена и залогирована!
🏃 View run third_experiment_neural_network at: http://127.0.0.1:8080/#/experiments/0/runs/8d1d38be345646f5bccb64306e83cff6
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_neural_network_third")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'attributes.run_name = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

model_path = f"{cfg_inference.model.artifacts_path}/{cfg_inference.model.model_file}"
tokenizer_path = f"preprocessing/{cfg_inference.model.tokenizer}"
encoder_path = f"preprocessing/{cfg_inference.model.encoder}"
info_path = f"preprocessing/{cfg_inference.model.preprocessing_info}"

print("Загружаем модели...")
try:
    model_file = client.download_artifacts(latest_run_model_id, model_path)
    tokenizer_file = client.download_artifacts(latest_run_model_id, tokenizer_path)
    encoder_file = client.download_artifacts(latest_run_model_id, encoder_path)
    info_file = client.download_artifacts(latest_run_model_id, info_path)
    print("Все модели успешно загружены")
except Exception as e:
    print(f"Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model_keras = tf.keras.models.load_model(model_file)

with open(tokenizer_file, 'rb') as f:
    tokenizer = pickle.load(f)

with open(encoder_file, 'rb') as f:
    encoder = pickle.load(f)

with open(info_file, 'rb') as f:
    preprocessing_info = pickle.load(f)

print("Все модели загружены в память")
print(f"Информация о предобработке: {preprocessing_info}")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

print("Выполняем предсказание...")

tokenized_text = tokenizer.texts_to_sequences([text])
print(f"Токенизированный текст: {tokenized_text}")

max_len = preprocessing_info['max_sequence_length']
print(f"Максимальная длина последовательности: {max_len}")

padded_text = pad_sequences(tokenized_text, maxlen=max_len, padding='post', truncating='post')
print(f"Текст после паддинга: {padded_text}")

pred = model_keras.predict(padded_text)
print(f"Сырые предсказания: {pred}")

predicted_class_ind = np.argmax(pred, axis=1)
predicted_class = encoder.inverse_transform(predicted_class_ind)
confidence = np.max(pred, axis=1)[0]

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_class[0]}')
print(f'Уверенность: {confidence:.4f}')
print(f'Все вероятности:')
for i, prob in enumerate(pred[0]):
    class_name = encoder.inverse_transform([i])[0]
    print(f'  {class_name}: {prob:.4f}')
print('=' * 100)

Run ID модели: 8d1d38be345646f5bccb64306e83cff6
Загружаем модели...


✅ Все модели успешно загружены
Загружаем модели в память...
✅ Все модели загружены в память
📊 Информация о предобработке: {'vocab_size': 5376, 'max_sequence_length': 85, 'num_classes': 10, 'classes': ['O', 'ВКУС_NEGATIVE', 'ВКУС_NEUTRAL', 'ВКУС_POSITIVE', 'ПАЧКА_NEGATIVE', 'ПАЧКА_NEUTRAL', 'ПАЧКА_POSITIVE', 'ТЕКСТУРА_NEGATIVE', 'ТЕКСТУРА_NEUTRAL', 'ТЕКСТУРА_POSITIVE']}
Выполняем предсказание...
Токенизированный текст: [[43, 46, 1532, 5, 2, 1]]
Максимальная длина последовательности: 85
Текст после паддинга: [[  43   46 1532    5    2    1    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step
Сырые пр

## Третий эксперимент (Трансформеры)

### Первый датасет

In [ ]:
cfg = load_config("transformers_first")

with mlflow.start_run(run_name='transformers_experiment_1'):
    
    mlflow.set_tag('Transformers', cfg.model.version)
    client = MlflowClient()

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================
    

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"Найден run с датасетом: {dataset_run_id}")


    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art_loc = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art_loc)
        print(f"Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"Ошибка загрузки: {e}")

        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "First_version.csv"
        ]
        
        for path in alternative_paths:
            try:
                art_loc = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art_loc)
                print(f"Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")


    df = df[[cfg.data.text_column, cfg.data.label_column]]
    df = df.rename(columns={cfg.data.label_column: "labels"})
    display(df)

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ДАТАСЕТА/ТАРГЕТОВ
    # =====================================================================================================================================
    
    df_train, df_temp = train_test_split(
        df, 
        test_size=cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['labels']
    )
    
    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=df_temp['labels']
    )

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(df_train['labels'])
    y_test = encoder.transform(df_test['labels'])
    y_val = encoder.transform(df_val['labels'])

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА МОДЕЛИ
    # =====================================================================================================================================
    
    mlflow.transformers.autolog()


    tokenizer = AutoTokenizer.from_pretrained(cfg.model.model_name)

    model = AutoModelForSequenceClassification.from_pretrained(
        cfg.model.model_name, 
        num_labels=len(encoder.classes_), 
        id2label={i: label for i, label in enumerate(encoder.classes_)},
        label2id={label: i for i, label in enumerate(encoder.classes_)}
    )
    

    dataset_train = Dataset.from_pandas(df_train.assign(labels=y_train))    
    dataset_test = Dataset.from_pandas(df_test.assign(labels=y_test))    
    dataset_val = Dataset.from_pandas(df_val.assign(labels=y_val))


    def tokenize_dataset(row):
        tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
        return tokenizer(row[cfg.data.text_column], **tokenizer_config)

    dataset_tokenized_train = dataset_train.map(tokenize_dataset, batched=False)
    dataset_tokenized_test = dataset_test.map(tokenize_dataset, batched=False)
    dataset_tokenized_val = dataset_val.map(tokenize_dataset, batched=False)


    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding=True,
        return_tensors='pt'
    )
    

    training_args = TrainingArguments(
        output_dir=cfg.training.output_dir,
        overwrite_output_dir=True,
        logging_dir='./logs/',
        num_train_epochs=cfg.training.num_train_epochs,
        learning_rate=cfg.training.learning_rate,
        per_device_train_batch_size=cfg.training.per_device_train_batch_size,
        per_device_eval_batch_size=cfg.training.per_device_eval_batch_size,
        eval_strategy=cfg.training.eval_strategy,
        save_strategy=cfg.training.save_strategy,
        warmup_ratio=cfg.training.warmup_ratio,
        lr_scheduler_type=cfg.training.lr_scheduler_type,
        metric_for_best_model=cfg.training.metric_for_best_model,
        weight_decay=cfg.training.weight_decay,
        load_best_model_at_end=cfg.training.load_best_model_at_end,
        save_total_limit=cfg.training.save_total_limit,
        max_grad_norm=cfg.training.max_grad_norm,
        logging_steps=cfg.training.logging_steps
    )


    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        
        return {
            'f1-score': f1_score(labels, predictions, average='weighted'),
            'accuracy': accuracy_score(labels, predictions)
        }


    trainer = Trainer(
        model=model, 
        args=training_args, 
        train_dataset=dataset_tokenized_train,
        eval_dataset=dataset_tokenized_val,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=cfg.training.early_stopping_patience)]
    )


    trainer.train()


    final_metrics = trainer.evaluate(dataset_tokenized_test)
    print(f"Финальные метрики: {final_metrics}")


    # =====================================================================================================================================
    #                                         СОХРАНЕНИЕ И ЛОГИРОВАНИЕ МОДЕЛИ
    # =====================================================================================================================================

    final_predictions = trainer.predict(dataset_tokenized_test)
    predictions = np.argmax(final_predictions.predictions, axis=1)
    labels = final_predictions.label_ids

    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    
    mlflow.log_metric('f1_score', f1)
    mlflow.log_metric('accuracy', accuracy) 
    
    for key, value in final_predictions.metrics.items():
        if key.startswith('eval_'):
            clean_key = key.replace('eval_', '')
            mlflow.log_metric(clean_key, value)

    print(f"Final metrics - F1: {f1:.4f}, Accuracy: {accuracy:.4f}, Loss: {test_loss:.4f}")

    model_dir = cfg.artifacts.model_dir
    tokenizer_dir = cfg.artifacts.tokenizer_dir

    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(tokenizer_dir, exist_ok=True)

    model.save_pretrained(model_dir)
    tokenizer.save_pretrained(tokenizer_dir)

    mlflow.log_artifacts(model_dir, "model")
    mlflow.log_artifacts(tokenizer_dir, "tokenizer")

    with open('label_encoder.pkl', 'wb') as f:
        pickle.dump(encoder, f)
    mlflow.log_artifact('label_encoder.pkl')

    mlflow.log_params({
        'model_name': cfg.model.model_name,
        'num_labels': len(encoder.classes_),
        'num_train_epochs': cfg.training.num_train_epochs,
        'learning_rate': cfg.training.learning_rate,
        'batch_size': cfg.training.per_device_train_batch_size,
        'early_stopping_patience': cfg.training.early_stopping_patience
    })

    print("Трансформер успешно обучен и залогирован!")

✅ Найден run с датасетом: 8690112afeac4c7faf0867c5d30d2dd9


✅ Датасет загружен: datasets/first_experiment_dataset.csv


,span,labels
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2503,ярко красиво фиолетовый цвет любить новогодний...,ПАЧКА_POSITIVE
2504,чипсы довольно жирный не критично заметно,ТЕКСТУРА_NEUTRAL
2505,сам чипсы не идеально круглый,ТЕКСТУРА_NEUTRAL
2506,красивый золотистый нет пригорелый поломать ми...,ТЕКСТУРА_POSITIVE


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 251/251 [00:00<00:00, 2711.61 examples/s]
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,F1-score,Accuracy
1,2.251500,2.160471,0.069781,0.203187
2,2.012900,1.942594,0.267171,0.346614
3,1.850000,1.767167,0.336707,0.398406
4,1.694400,1.668675,0.373185,0.438247
5,1.613000,1.593321,0.389435,0.454183
6,1.563400,1.545394,0.426005,0.486056
7,1.432900,1.516056,0.445592,0.501992
8,1.399200,1.491220,0.445530,0.501992
9,1.422100,1.481730,0.450813,0.505976
10,1.402100,1.477343,0.453505,0.509960


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pin

✅ Финальные метрики: {'eval_loss': 1.4958806037902832, 'eval_f1-score': 0.44198462104907194, 'eval_accuracy': 0.5059760956175299, 'eval_runtime': 0.416, 'eval_samples_per_second': 603.431, 'eval_steps_per_second': 38.466, 'epoch': 10.0}


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


✅ Final metrics - F1: 0.4420, Accuracy: 0.5060, Loss: 3.4085
✅ Трансформер успешно обучен и залогирован!
🏃 View run transformers_experiment_1 at: http://127.0.0.1:8080/#/experiments/0/runs/82086ed0872f47c58376be0d105fed8f
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результата

In [ ]:
cfg_inference = load_config("inference_transformers_first")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================


latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'tags.mlflow.runName = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")


print("Загружаем модель и токенизатор...")
try:

    model_dir = client.download_artifacts(latest_run_model_id, cfg_inference.model.artifacts_path)
    tokenizer_dir = client.download_artifacts(latest_run_model_id, cfg_inference.model.tokenizer_path)
    print("✅ Модель и токенизатор успешно загружены")
except Exception as e:
    print(f"❌ Ошибка загрузки моделей: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

print("Модель и токенизатор загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================


text = cfg_inference.test_text

print("Выполняем предсказание...")


tokenizer_config = {k: v for k, v in cfg_inference.tokenizer.items() if v is not None}
inputs = tokenizer(text, **tokenizer_config)


with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

predicted_class_idx = predictions.argmax().item()
predicted_prob = predictions.max().item()

id2label = model.config.id2label
predicted_label = id2label[predicted_class_idx]

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_label}')
print(f'Вероятность: {predicted_prob:.4f}')
print(f'Все вероятности:')
for i, prob in enumerate(predictions[0]):
    label = id2label[i]
    print(f'  {label}: {prob:.4f}')
print('=' * 100)

Run ID модели: 82086ed0872f47c58376be0d105fed8f
Загружаем модель и токенизатор...


✅ Модель и токенизатор успешно загружены
Загружаем модели в память...
✅ Модель и токенизатор загружены в память
Выполняем предсказание...
Исходный текст: "В целом, чипсы Ашан Красная птица Барбекю вполне съедобные"
Предсказанная метка: ВКУС_POSITIVE
Вероятность: 0.3161
Все вероятности:
  O: 0.1456
  ВКУС_NEGATIVE: 0.1105
  ВКУС_NEUTRAL: 0.1792
  ВКУС_POSITIVE: 0.3161
  ПАЧКА_NEGATIVE: 0.0258
  ПАЧКА_NEUTRAL: 0.0288
  ПАЧКА_POSITIVE: 0.0389
  ТЕКСТУРА_NEGATIVE: 0.0373
  ТЕКСТУРА_NEUTRAL: 0.0438
  ТЕКСТУРА_POSITIVE: 0.0740


### Второй датасет

In [ ]:
cfg = load_config("transformers_second")

with mlflow.start_run(run_name='transformers_experiment_2'):
    
    mlflow.set_tag('Transformers', cfg.model.version)
    client = MlflowClient()

    # =====================================================================================================================================
    #                                         СЧИТЫВАНИЕ ДАТАСЕТА
    # =====================================================================================================================================

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_id],
        filter_string=f"tags.mlflow.runName = '{cfg.data.source_run}'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError(f"Не найден run: {cfg.data.source_run}")

    dataset_run = dataset_runs[0]
    dataset_run_id = dataset_run.info.run_id
    print(f"Найден run с датасетом: {dataset_run_id}")

    try:
        dataset_path = f"{cfg.data.dataset_path}/{cfg.data.dataset_file}"
        art_loc = client.download_artifacts(dataset_run_id, dataset_path)
        df = pd.read_csv(art_loc)
        print(f"Датасет загружен: {dataset_path}")
    except Exception as e:
        print(f"Ошибка загрузки: {e}")

        alternative_paths = [
            cfg.data.dataset_file,
            f"artifacts/{cfg.data.dataset_path}/{cfg.data.dataset_file}",
            "second_experiment_dataset.csv"
        ]
        
        for path in alternative_paths:
            try:
                art_loc = client.download_artifacts(dataset_run_id, path)
                df = pd.read_csv(art_loc)
                print(f"Датасет загружен: {path}")
                break
            except:
                continue
        else:
            raise ValueError("Не удалось загрузить датасет")

    display(df.head())

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА ДАННЫХ
    # =====================================================================================================================================

    encoder = LabelEncoder()
    df['label_encoded'] = encoder.fit_transform(df[cfg.data.label_column])
    df = df.rename(columns={'label_encoded': 'labels'})
    df = df[[cfg.data.text_column, 'labels']]

    # =====================================================================================================================================
    #                                         ПОДГОТОВКА МОДЕЛИ
    # =====================================================================================================================================

    tokenizer = AutoTokenizer.from_pretrained(cfg.model.model_name)

    model = AutoModelForSequenceClassification.from_pretrained(
        cfg.model.model_name,
        num_labels=len(encoder.classes_),
        id2label={i: label for i, label in enumerate(encoder.classes_)},
        label2id={label: i for i, label in enumerate(encoder.classes_)}
    )
    
    # =====================================================================================================================================
    #                                         РАЗДЕЛЕНИЕ ДАННЫХ
    # =====================================================================================================================================

    df_train, df_temp = train_test_split(
        df, 
        train_size=1-cfg.training.test_size, 
        random_state=cfg.training.random_state, 
        stratify=df['labels']
    )
    
    df_test, df_val = train_test_split(
        df_temp, 
        test_size=cfg.training.val_size, 
        random_state=cfg.training.random_state, 
        stratify=df_temp['labels']
    )

    dataset_train = Dataset.from_pandas(df_train)
    dataset_test = Dataset.from_pandas(df_test)
    dataset_val = Dataset.from_pandas(df_val)

    # =====================================================================================================================================
    #                                         ТОКЕНИЗАЦИЯ
    # =====================================================================================================================================

    def tokenize_dataset(batch):
        tokenizer_config = {k: v for k, v in cfg.tokenizer.items() if v is not None}
        return tokenizer(
            batch[cfg.data.text_column],
            **tokenizer_config
        )

    tokenized_train = dataset_train.map(tokenize_dataset, batched=True)
    tokenized_test = dataset_test.map(tokenize_dataset, batched=True)
    tokenized_val = dataset_val.map(tokenize_dataset, batched=True)

    # =====================================================================================================================================
    #                                         DATA COLLATOR
    # =====================================================================================================================================

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding=True,
        max_length=cfg.tokenizer.max_length,
        return_tensors="pt"
    )

    # =====================================================================================================================================
    #                                         TRAINING ARGUMENTS
    # =====================================================================================================================================

    training_args = TrainingArguments(
        output_dir=cfg.training.output_dir,
        num_train_epochs=cfg.training.num_train_epochs,
        learning_rate=cfg.training.learning_rate,
        per_device_train_batch_size=cfg.training.per_device_train_batch_size,
        per_device_eval_batch_size=cfg.training.per_device_eval_batch_size,
        eval_strategy=cfg.training.eval_strategy,
        save_strategy=cfg.training.save_strategy,
        logging_dir=cfg.training.logging_dir,
        logging_steps=cfg.training.logging_steps,
        load_best_model_at_end=cfg.training.load_best_model_at_end,
        metric_for_best_model=cfg.training.metric_for_best_model
    )

    # =====================================================================================================================================
    #                                         МЕТРИКИ
    # =====================================================================================================================================

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        y_pred = np.argmax(predictions, axis=1)

        return {
            'f1_score': f1_score(y_pred, labels, average='weighted'),
            'accuracy': accuracy_score(y_pred, labels)
        }
    
    # =====================================================================================================================================
    #                                         TRAINER И ОБУЧЕНИЕ
    # =====================================================================================================================================

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
    )

    trainer.train()

    # =====================================================================================================================================
    #                                         ОЦЕНКА И ЛОГИРОВАНИЕ МЕТРИК
    # =====================================================================================================================================

    test_predictions = trainer.predict(tokenized_test)
    test_preds = np.argmax(test_predictions.predictions, axis=1)
    test_labels = test_predictions.label_ids

    accuracy = accuracy_score(test_labels, test_preds)
    f1_score_value = f1_score(test_labels, test_preds, average='weighted')

    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_score", f1_score_value)

    print(f"🎯 Финальные метрики:")
    print(f"   accuracy: {accuracy:.4f}")
    print(f"   f1_score: {f1_score_value:.4f}")

    # =====================================================================================================================================
    #                                         СОХРАНЕНИЕ АРТЕФАКТОВ
    # =====================================================================================================================================

    model_dir = cfg.artifacts.model_dir
    tokenizer_dir = cfg.artifacts.tokenizer_dir
    
    import shutil
    if os.path.exists(model_dir):
        shutil.rmtree(model_dir)
    if os.path.exists(tokenizer_dir):
        shutil.rmtree(tokenizer_dir)

    trainer.save_model(model_dir)
    tokenizer.save_pretrained(tokenizer_dir)

    with open('label_encoder.pkl', 'wb') as f:
        pickle.dump(encoder, f)

    mlflow.log_artifacts(model_dir, "model")
    mlflow.log_artifacts(tokenizer_dir, "tokenizer")
    mlflow.log_artifact('label_encoder.pkl')

    # =====================================================================================================================================
    #                                         ЛОГИРОВАНИЕ ПАРАМЕТРОВ
    # =====================================================================================================================================

    mlflow.log_params({
        'model_name': cfg.model.model_name,
        'num_labels': len(encoder.classes_),
        'num_train_epochs': cfg.training.num_train_epochs,
        'learning_rate': cfg.training.learning_rate,
        'batch_size': cfg.training.per_device_train_batch_size,
        'test_size': cfg.training.test_size,
        'val_size': cfg.training.val_size
    })

    # =====================================================================================================================================
    #                                         ОЧИСТКА ВРЕМЕННЫХ ФАЙЛОВ
    # =====================================================================================================================================

    shutil.rmtree(model_dir)
    shutil.rmtree(tokenizer_dir)
    os.remove('label_encoder.pkl')

    print("Трансформер успешно обучен и залогирован!")

✅ Найден run с датасетом: ba68188717234d8ea8f01745d4816ea8


✅ Датасет загружен: datasets/second_experiment_dataset.csv


,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 251/251 [00:00<00:00, 6745.76 examples/s]
C:\Users\Smart\AppData\Local\Temp\ipykernel_160\4146632116.py:161: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2863: UserWarning: `max_length

Epoch,Training Loss,Validation Loss,F1 Score,Accuracy
1,2.076000,2.079241,0.366077,0.270916
2,1.914100,1.852924,0.464990,0.394422
3,1.721000,1.722221,0.488090,0.422311
4,1.565800,1.661844,0.478570,0.418327
5,1.465800,1.595550,0.531171,0.462151
6,1.469300,1.566464,0.516893,0.454183
7,1.430800,1.547074,0.504289,0.446215
8,1.326700,1.541347,0.499597,0.442231
9,1.311400,1.532064,0.498807,0.442231
10,1.347100,1.527224,0.498972,0.446215


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2863: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2863: UserWarning: `max_length` is ignored when `padding`=`True

🎯 Финальные метрики:
   accuracy: 0.4422
   f1_score: 0.3732
✅ Трансформер успешно обучен и залогирован!
🏃 View run transformers_experiment_2 at: http://127.0.0.1:8080/#/experiments/0/runs/734f31b8cd0c48ada580eb519ae9b457
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов

In [ ]:
cfg_inference = load_config("inference_transformers_second")

client = MlflowClient()

# ===========================================================================================
# ПОЛУЧЕНИЕ МОДЕЛЕЙ
# ===========================================================================================

latest_run_model = client.search_runs(
    experiment_ids=[cfg_inference.mlflow.experiment_id],
    filter_string=f'attributes.run_name = "{cfg_inference.model.run_name}"',
    order_by=['attributes.end_time desc']
)

if not latest_run_model:
    raise ValueError(f"Не найден run: {cfg_inference.model.run_name}")

latest_run_model_id = latest_run_model[0].info.run_id
print(f"Run ID модели: {latest_run_model_id}")

print("Загружаем модель, токенизатор и энкодер...")
try:
    model_dir = client.download_artifacts(latest_run_model_id, 'model')
    tokenizer_dir = client.download_artifacts(latest_run_model_id, 'tokenizer')
    encoder_path = client.download_artifacts(latest_run_model_id, 'label_encoder.pkl')
    print("Модель, токенизатор и энкодер успешно загружены")
except Exception as e:
    print(f"Ошибка загрузки: {e}")
    raise

# ===========================================================================================
# ЗАГРУЗКА И ИСПОЛЬЗОВАНИЕ МОДЕЛЕЙ
# ===========================================================================================

print("Загружаем модели в память...")
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

with open(encoder_path, 'rb') as f:
    encoder = pickle.load(f)

print("Модель, токенизатор и энкодер загружены в память")

# ===========================================================================================
# ПРЕДСКАЗАНИЕ
# ===========================================================================================

text = cfg_inference.test_text

print("Выполняем предсказание...")

tokenizer_config = {k: v for k, v in cfg_inference.tokenizer.items() if v is not None}
inputs = tokenizer(text, **tokenizer_config)

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

predicted_class_idx = predictions.argmax().item()
predicted_prob = predictions.max().item()

predicted_label = encoder.inverse_transform([predicted_class_idx])[0]

print('=' * 100)
print(f'Исходный текст: "{text}"')
print(f'Предсказанная метка: {predicted_label}')
print(f'Вероятность: {predicted_prob:.4f}')
print(f'Все вероятности:')
for i, prob in enumerate(predictions[0]):
    label = encoder.inverse_transform([i])[0]
    print(f'  {label}: {prob:.4f}')
print('=' * 100)

Run ID модели: 734f31b8cd0c48ada580eb519ae9b457
Загружаем модель, токенизатор и энкодер...


✅ Модель, токенизатор и энкодер успешно загружены
Загружаем модели в память...
✅ Модель, токенизатор и энкодер загружены в память
Выполняем предсказание...
Исходный текст: "Пример текста для классификации на русском языке"
Предсказанная метка: ПАЧКА_NEUTRAL
Вероятность: 0.2678
Все вероятности:
  O: 0.0901
  ВКУС_NEGATIVE: 0.0184
  ВКУС_NEUTRAL: 0.0321
  ВКУС_POSITIVE: 0.0212
  ПАЧКА_NEGATIVE: 0.0978
  ПАЧКА_NEUTRAL: 0.2678
  ПАЧКА_POSITIVE: 0.1886
  ТЕКСТУРА_NEGATIVE: 0.0811
  ТЕКСТУРА_NEUTRAL: 0.1246
  ТЕКСТУРА_POSITIVE: 0.0784


### Третий датасет

In [ ]:
cfg = load_config("transformers_third")

with mlflow.start_run(run_name=cfg.mlflow.run_name):

    client = MlflowClient()

    mlflow.set_tag("Dataset_version", cfg.mlflow.dataset_version)
    mlflow.log_param("model_name", cfg.model.name)

    # ---------------------------
    #   Загружаем датасет
    # ---------------------------

    dataset_runs = client.search_runs(
        experiment_ids=[cfg.mlflow.experiment_name], 
        filter_string=f"tags.mlflow.runName = 'Third dataset'",
        order_by=['attributes.end_time desc']
    )

    if not dataset_runs:
        raise ValueError("Не найден run с датасетом 'Third dataset'")

    dataset_run_id = dataset_runs[0].info.run_id
    print(f"Найден run с датасетом: {dataset_run_id}")

    dataset_path_in_run = "datasets/third_experiment_dataset.csv" 
    artifact_local_path = client.download_artifacts(dataset_run_id, dataset_path_in_run)

    df = pd.read_csv(artifact_local_path)
    print(f"Датасет загружен из run: {artifact_local_path}")

    # ---------------------------
    #   ПОДГОТОВКА ДАННЫХ
    # ---------------------------

    encoder = LabelEncoder()
    df["labels"] = encoder.fit_transform(df["label"])
    df["labels"] = df["labels"].astype(int)

    mlflow.log_param("num_labels", len(encoder.classes_))

    df_train, df_temp = train_test_split(
        df,
        test_size=1 - cfg.data.train_size,
        random_state=cfg.data.random_state,
        shuffle=True,
        stratify=df["labels"]
    )

    df_val, df_test = train_test_split(
        df_temp,
        test_size=cfg.data.val_size,
        random_state=cfg.data.random_state,
        shuffle=True,
        stratify=df_temp["labels"]
    )

    print(f"Размеры данных: Train={len(df_train)}, Val={len(df_val)}, Test={len(df_test)}")

    # ---------------------------
    #   ПОДГОТОВКА ДАТАСЕТОВ (ИСПРАВЛЕННАЯ)
    # ---------------------------

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    dataset_train = Dataset.from_dict({
        "text": df_train["span"].tolist(),
        "labels": df_train["labels"].tolist()
    })
    
    dataset_val = Dataset.from_dict({
        "text": df_val["span"].tolist(), 
        "labels": df_val["labels"].tolist()
    })
    
    dataset_test = Dataset.from_dict({
        "text": df_test["span"].tolist(),
        "labels": df_test["labels"].tolist()
    })

    # ---------------------------
    #   ТОКЕНИЗАЦИЯ
    # ---------------------------

    tokenizer = AutoTokenizer.from_pretrained(cfg.model.name)

    def tokenize_function(examples):
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            padding=False,
            max_length=cfg.model.max_length
        )
        return tokenized

    tokenized_train = dataset_train.map(tokenize_function, batched=True)
    tokenized_val = dataset_val.map(tokenize_function, batched=True)
    tokenized_test = dataset_test.map(tokenize_function, batched=True)

    tokenized_train = tokenized_train.remove_columns(['text'])
    tokenized_val = tokenized_val.remove_columns(['text'])
    tokenized_test = tokenized_test.remove_columns(['text'])

    print("Проверка структуры данных после токенизации:")
    print(f"Train columns: {tokenized_train.column_names}")
    print(f"Sample train: {tokenized_train[0]}")

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding=True,
        max_length=cfg.model.max_length,
        return_tensors="pt"
    )

    # ---------------------------
    #   ИНИЦИАЛИЗАЦИЯ МОДЕЛИ
    # ---------------------------

    num_labels = len(encoder.classes_)

    model = AutoModelForSequenceClassification.from_pretrained(
        cfg.model.name,
        num_labels=num_labels,
        id2label={i: label for i, label in enumerate(encoder.classes_)},
        label2id={label: i for i, label in enumerate(encoder.classes_)},
        hidden_dropout_prob=cfg.model.dropout.hidden,
        attention_probs_dropout_prob=cfg.model.dropout.attention,
        classifier_dropout=cfg.model.dropout.classifier
    )

    # ---------------------------
    #   TRAINING ARGUMENTS
    # ---------------------------

    training_args = TrainingArguments(
        output_dir=cfg.training.output_dir,
        num_train_epochs=cfg.training.num_train_epochs,
        learning_rate=cfg.training.learning_rate,
        per_device_train_batch_size=cfg.training.per_device_train_batch_size,
        per_device_eval_batch_size=cfg.training.per_device_eval_batch_size,
        warmup_ratio=cfg.training.warmup_ratio,
        lr_scheduler_type=cfg.training.lr_scheduler_type,
        eval_strategy=cfg.training.eval_strategy,
        save_strategy=cfg.training.save_strategy,
        weight_decay=cfg.training.weight_decay,
        logging_steps=cfg.training.logging_steps,
        save_total_limit=cfg.training.save_total_limit,
        max_grad_norm=cfg.training.max_grad_norm,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        dataloader_pin_memory=False,
        dataloader_num_workers=0,
        remove_unused_columns=True
    )

    # ---------------------------
    #   МЕТРИКИ
    # ---------------------------

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=1)

        return {
            "f1": f1_score(labels, preds, average="weighted"),
            "accuracy": accuracy_score(labels, preds)
        }

    # ---------------------------
    #   ОБУЧЕНИЕ
    # ---------------------------

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=cfg.training.early_stopping_patience)]
    )

    print("🚀 Начинаем обучение...")
    trainer.train()

    # ---------------------------
    #   ОЦЕНКА И ЛОГИРОВАНИЕ
    # ---------------------------
    
    print("Оцениваем модель на тестовом наборе...")
    test_predictions = trainer.predict(tokenized_test)
    test_preds = np.argmax(test_predictions.predictions, axis=1)
    test_labels = test_predictions.label_ids
    
    test_f1 = f1_score(test_labels, test_preds, average="weighted")
    test_accuracy = accuracy_score(test_labels, test_preds)
    
    mlflow.log_metric("f1_score", test_f1)
    mlflow.log_metric("accuracy", test_accuracy)
    
    print(f"Финальные метрики на тесте:")
    print(f"F1-score: {test_f1:.4f}")
    print(f"Accuracy: {test_accuracy:.4f}")

    # ---------------------------
    #   СОХРАНЕНИЕ
    # ---------------------------

    model_dir = "best_transformer_model"
    tokenizer_dir = "best_transformer_tokenizer"

    trainer.save_model(model_dir)
    tokenizer.save_pretrained(tokenizer_dir)

    mlflow.log_artifacts(model_dir, artifact_path="model")
    mlflow.log_artifacts(tokenizer_dir, artifact_path="tokenizer")

    with open('label_encoder.pkl', 'wb') as f:
        pickle.dump(encoder, f)
    mlflow.log_artifact('label_encoder.pkl', artifact_path="preprocessing")

    import shutil
    shutil.rmtree(model_dir)
    shutil.rmtree(tokenizer_dir)
    os.remove('label_encoder.pkl')

    print("Трансформер успешно обучен и залогирован!")

✅ Найден run с датасетом: 327f7f34ba914fce83ca6c0e787e8f15


✅ Датасет загружен из run: C:\Users\Smart\AppData\Local\Temp\tmpkmxff0ir\third_experiment_dataset.csv
📊 Размеры данных: Train=3216, Val=402, Test=402


Map: 100%|██████████| 402/402 [00:00<00:00, 8308.13 examples/s]


🔍 Проверка структуры данных после токенизации:
Train columns: ['labels', 'input_ids', 'token_type_ids', 'attention_mask']
Sample train: {'labels': 3, 'input_ids': [101, 110, 55795, 37312, 107, 54557, 71854, 14236, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,1.299100,1.166594,0.591704,0.629353
2,0.675400,0.634449,0.794025,0.796020
3,0.339600,0.540969,0.832404,0.833333
4,0.102700,0.585891,0.849085,0.853234
5,0.081400,0.643048,0.857232,0.858209
6,0.068900,0.622935,0.869776,0.870647
7,0.030800,0.679291,0.867847,0.868159
8,0.023900,0.656416,0.868577,0.870647


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2863: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2863: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2863: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2863: Use

📊 Оцениваем модель на тестовом наборе...


c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2863: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


🎯 Финальные метрики на тесте:
   F1-score: 0.8191
   Accuracy: 0.8259
✅ Трансформер успешно обучен и залогирован!
🏃 View run transformers_experiment_3 at: http://127.0.0.1:8080/#/experiments/0/runs/948c02959d5a46b798a981a964d40f83
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


#### Проверка результатов (инференс)

In [ ]:
cfg = load_config("inference_transformers_third")

client = MlflowClient(tracking_uri=cfg.mlflow.tracking_uri)

# ================================
# 2. Ищем нужный run в MLflow
# ================================
print("Ищем run с моделью...")

runs = client.search_runs(
    experiment_ids=[cfg.mlflow.experiment_id],
    filter_string=f'attributes.run_name = "{cfg.model.run_name}"',
    order_by=["attributes.end_time desc"]
)

if not runs:
    raise ValueError(f"Run '{cfg.model.run_name}' не найден!")

run_id = runs[0].info.run_id
print(f"Найден run: {run_id}")

# ================================
# 3. Скачиваем артефакты
# ================================
print("Скачиваем модель, токенизатор и энкодер...")

try:
    model_dir = client.download_artifacts(run_id, cfg.model.artifacts_path)
    tokenizer_dir = client.download_artifacts(run_id, cfg.model.tokenizer_path)
    encoder_path = client.download_artifacts(run_id, "preprocessing/label_encoder.pkl")
    
    print(f"Модель скачана: {model_dir}")
    print(f"Токенизатор скачан: {tokenizer_dir}")
    print(f"Энкодер скачан: {encoder_path}")
    
except Exception as e:
    print(f"Ошибка загрузки артефактов: {e}")
    raise

# ================================
# 4. Загружаем модель в память
# ================================
print("Загружаем модель и компоненты...")

model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

with open(encoder_path, 'rb') as f:
    encoder = pickle.load(f)

print("Все компоненты загружены")

# ================================
# 5. Предсказание
# ================================
text = cfg.test_text
print(f"Тестовый текст: «{text}»")

tokenizer_cfg = {k: v for k, v in cfg.tokenizer.items() if v is not None}

inputs = tokenizer(text, **tokenizer_cfg)

with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

pred_idx = probs.argmax().item()
pred_prob = probs[0][pred_idx].item()

pred_label = encoder.inverse_transform([pred_idx])[0]

print("\n" + "="*100)
print(f"Исходный текст: «{text}»")
print(f"Предсказанная метка: {pred_label}")
print(f"Вероятность: {pred_prob:.4f}")
print("\nВсе вероятности:")
for i, p in enumerate(probs[0]):
    label = encoder.inverse_transform([i])[0]
    print(f"  {label}: {p:.4f}")
print("="*100)

Ищем run с моделью...
✅ Найден run: 948c02959d5a46b798a981a964d40f83
Скачиваем модель, токенизатор и энкодер...


📁 Модель скачана: C:\Users\Smart\AppData\Local\Temp\tmp1ut_wjcm\model
📁 Токенизатор скачан: C:\Users\Smart\AppData\Local\Temp\tmpv5m85rm0\tokenizer
📁 Энкодер скачан: C:\Users\Smart\AppData\Local\Temp\tmpstx4jzoy\label_encoder.pkl
Загружаем модель и компоненты...


The tokenizer you are loading from 'C:\Users\Smart\AppData\Local\Temp\tmpv5m85rm0\tokenizer' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e.  This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ Все компоненты загружены
Тестовый текст: «Можно было сделать чипсы и повкуснее»

Исходный текст: «Можно было сделать чипсы и повкуснее»
Предсказанная метка: ВКУС_NEUTRAL
Вероятность: 0.7908

Все вероятности:
  O: 0.0010
  ВКУС_NEGATIVE: 0.0021
  ВКУС_NEUTRAL: 0.7908
  ВКУС_POSITIVE: 0.1538
  ПАЧКА_NEGATIVE: 0.0008
  ПАЧКА_NEUTRAL: 0.0012
  ПАЧКА_POSITIVE: 0.0004
  ТЕКСТУРА_NEGATIVE: 0.0018
  ТЕКСТУРА_NEUTRAL: 0.0021
  ТЕКСТУРА_POSITIVE: 0.0459


# Выбор лучшей модели (для инференса)

In [ ]:
client = MlflowClient()

runs = client.search_runs(experiment_ids = ['0'],
                          order_by = ['metrics.f1_score desc', 'metrics.accuracy desc'])

best_run = runs[0]


best_run_id = best_run.info.run_id

model_loc = client.download_artifacts(best_run_id, path='model')
tokenizer_loc = client.download_artifacts(best_run_id, path='tokenizer')

tokenizer = AutoTokenizer.from_pretrained(tokenizer_loc)
model = AutoModelForSequenceClassification.from_pretrained(model_loc)


model_final = pipeline('text-classification', model = model, tokenizer = tokenizer)

print(model_final(['Вкус просто бомбический']))



The tokenizer you are loading from 'C:\Users\Smart\AppData\Local\Temp\tmpgi_nhox8\tokenizer' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e.  This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cpu


[{'label': 'ВКУС_NEGATIVE', 'score': 0.9860121607780457}]


In [11]:
output_dir_model = './best_model/model'
output_dir_tokenizer = './best_model/tokenizer'

model.save_pretrained(output_dir_model)
tokenizer.save_pretrained(output_dir_tokenizer)

('./best_model/tokenizer\\tokenizer_config.json',
 './best_model/tokenizer\\special_tokens_map.json',
 './best_model/tokenizer\\vocab.txt',
 './best_model/tokenizer\\added_tokens.json',
 './best_model/tokenizer\\tokenizer.json')

In [16]:
model_test = AutoModelForSequenceClassification.from_pretrained('best_model\\model')
tokenizer_test = AutoTokenizer.from_pretrained('best_model\\tokenizer')


pipe = pipeline('text-classification', model = model_test, tokenizer = tokenizer_test)


print(pipe(['Чипсы не очень']))

The tokenizer you are loading from 'best_model\tokenizer' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e.  This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cpu


[{'label': 'ВКУС_NEGATIVE', 'score': 0.9977165460586548}]
